In [1]:
import sys 
import pandas as pd
import logging

sys.path.append('../')

import utils.llm 
import utils.scoring

logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

class Config:
    DATA_DIR = '../data/'
    DATA_OUT_DIR = '../data_out/'
    TEMPERATURE = 0.01 # responseを得るのを利用したtemperatureと同じ値に設定
    SAMPLE_N = 30
    RANDOM_STR_LENGTH = 600
    TARGET_QUIZ_RANGE = list(range(0, 201))
    
def build_prompt(query:str):
    
    import random
    
    # ランダム文字列を生成
    RANDOM_STR_LENGTH = Config.RANDOM_STR_LENGTH
    random_str = ''.join([random.choice('abcdefghijklmnopqrstuvwxyz') for i in range(RANDOM_STR_LENGTH)])
    
    prompt = f'{random_str} {query}'
    
    return prompt
    


quiz_df = pd.read_csv(Config.DATA_DIR + 'df_JAQKET.csv')

num_questions = quiz_df.shape[0]

# num_questions = 2


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
llm = utils.llm.LLM()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Loading binary /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No GPU detected! Check your CUDA paths. Proceeding to load CPU-only library...
  warn(msg)


In [3]:
def sampling(query:str,temperature:float,sample_n:int) -> list[str]:
    
    sampled = []
    
    for i in range(sample_n):
        
        q = build_prompt(query)
        sampled.append(llm.generate(q,temperature))
    
    logger.info("sampled: {}".format(sampled))
    
    return sampled


In [4]:
# serial_number, qid, query, responseを記録するdf

df = pd.DataFrame(columns=['serial_number','qid','query','sample'])

serial_number = 0

for i in Config.TARGET_QUIZ_RANGE:
    
    logger.info("processing {} th question".format(i))
    
    qid = quiz_df.iloc[i]['qid']
    query = quiz_df.iloc[i]['question']
    response = quiz_df.iloc[i]['generated_answer']
    
    sampled = sampling(query,Config.TEMPERATURE,Config.SAMPLE_N)
    
    for sample in sampled:
        
        #  The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
        
        df = pd.concat([df,pd.DataFrame([[serial_number,qid,query,sample]],columns=['serial_number','qid','query','sample'])])
        
        serial_number += 1


processing 0 th question


sampled: ['北極海産貝類', 'マンタ', '北極海産貝類', 'イルカ', 'マンタ', 'マンタ', 'マンタ', 'パッセージ内の関連する文は、「ftonndmgqrwherfnmjgsxuspbvyhyhzxuspbongkengzxyyhjklopenfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorfjorf', 'ウミガメ', 'マンタ', '北極海産貝類', 'パッセージ中の関連する文は、「pfjikxwqrixpkomtakexwlbrirbtgdcuyqxgedgworlbrirbtgdcuyqxgedgworlbrirbtgdcuyqxgedgwormsfzpagewihuwwmhkxmzkhcucvxklaxocvzrrtcvfwmyduzptmxiykxwmhkxmzkhcucvxlullymg', 'マンタ', 'マンタ', 'マンタ', 'ウミガメ', 'マンタ', 'マンタ', 'マンタ', 'ウミガメ', '北極海産貝類', 'マンタ', 'マンタ', 'マンタ', '北極海産貝類', 'イルカ', 'ウミガメ', 'ウミガメ', 'マンタ', 'ウミガメ']
processing 1 th question
sampled: ['『東方見聞録』。マルコ・ポーロは、1453年に生まれ、1547年に亡くなりました。ルスティケロは、イタリアのフィレンツェ出身の作家です。彼は、イタリアやフランスなどの国々を旅して、多くの旅行記や歴史書を書きました。これらの作品は、ヨーロッパやアジアの国々に広く読まれています。マルコ・ポーロは、中国や日本を訪れたことでも有名です。ルスティケロは、中国や日本の文化や習慣を描写した多数の著作を残しています。マルコ・ポーロは、東洋の芸術や文化に大きな影響を与え、今日でも高く評価されています。', '『東方見聞録』。 マルコ・ポーロは、1453年に生まれ、1547年に亡くなりました。 『東方見聞録』は、ヴェネツィア出身の貿易商によって書かれました。 ヴェネツィアは地中海に位置しています。 マルコ・ポーロは、イタリアに旅行し、ヴェネツィアや近隣の島々を

In [5]:
df.shape

(6030, 4)

In [6]:
import datetime

info = 'temperature_{}_sample_n_{}_random_str_length_{}'.format(Config.TEMPERATURE,Config.SAMPLE_N,Config.RANDOM_STR_LENGTH)
suffix = datetime.datetime.now().strftime('%m_%d_%H_%M_%S')

df.to_csv(Config.DATA_OUT_DIR + 'random_query_{}_{}.csv'.format(info,suffix),index=False)




In [7]:
df

,serial_number,qid,query,sample
0,0,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,北極海産貝類
0,1,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,マンタ
0,2,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,北極海産貝類
0,3,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,イルカ
0,4,QA20CAPR-0001,和名をハダカカメガイといい、実は巻き貝の一種とされている、その姿から「流氷の天使」と呼ばれる...,マンタ
...,...,...,...,...
0,6025,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦 イネ科 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 ...
0,6026,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦 イネ科 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 ...
0,6027,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米科 イネ科 米科 米科 米科 米科 米科 米科 米科 米科 米科 米科 米科 米科 米科 ...
0,6028,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦 イネ科 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 米麦 ...
